In [1]:
import sys
sys.path.append('../')

In [2]:
import pickle

from pytorch_lightning import Trainer
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

from torchts.core.spatiotemporal.dcrnn_model import count_parameters
from torchts.nn.models.seq2seq import Decoder, Encoder, Seq2Seq

In [3]:
def scale_data(data_x, data_y, out_pos = 0, return_current_avg_std = False):
    """ 
    Arg:
        data_x: features
        data_y: labels
        out_pos: the position of feature of which average and stand deviation will be returned.
    returns:
        1. Normalized features and labels
        2. Average and standard deviation of the selected feature.
    """
    avg = data_x[:,:,out_pos].mean()
    std = data_x[:,:,out_pos].std()
#     c_avg = data_x[:,:,1].mean()
#     c_std = data_x[:,:,1].std()
    for i in range(data_x.shape[-1]):
        data_x[:,:,i] = (data_x[:,:,i] - data_x[:,:,i].mean())/data_x[:,:,i].std()
    data_y = (data_y-avg)/std
    if return_current_avg_std:
        return data_x, data_y, (avg, std)  
#         return data_x, data_y, (avg, std), (c_avg, c_std)   
    else:
        return data_x, data_y, (avg, std)

In [4]:
filename = 'traffic_bayArea_station_400001.pkl'

with open(filename, "rb") as fout:
# with open(join(base_dir, filename), "rb") as fout:
    c_time_series = pickle.load(fout)

In [5]:
sample_size = c_time_series.shape[0]
segment_size = c_time_series.shape[1]
pred_size = int(segment_size/2)

test_size = sample_size // 5
train_valid_size = test_size * 4
training_size = test_size * 7//2
validation_size = test_size * 1//2

In [6]:
X_all = c_time_series[:train_valid_size+test_size,:pred_size,:]
Y_all = c_time_series[:train_valid_size+test_size,pred_size:,:]

X, Y, (avg, std) = scale_data(X_all, Y_all, out_pos = 0, return_current_avg_std = True)
X, Y = X.float(), Y.float()

In [7]:
learning_rate = 0.01
dropout_rate = 0.6
num_layers = 1
hidden_dim = 128

input_steps = segment_size
output_steps = segment_size
input_size = 1
output_size = 1

train_idx = list(range(training_size))
valid_idx = list(range(training_size, train_valid_size))
test_idx = list(range(train_valid_size, train_valid_size + test_size))

In [8]:
batch_size = 64

train_set = TensorDataset(X[train_idx], Y[train_idx])
valid_set = TensorDataset(X[valid_idx], Y[valid_idx])
test_set = TensorDataset(X[test_idx], Y[test_idx])

train_generator = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_generator = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_generator = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [9]:
d = list(train_generator)

In [10]:
input_tensor = d[0][0]
target_tensor = d[0][1]

In [11]:
horizon = target_tensor.shape[1]
output_dim = target_tensor.shape[-1]

encoder = Encoder(input_size, hidden_dim, num_layers, dropout_rate)
decoder = Decoder(output_size, hidden_dim, num_layers, dropout_rate)

model = Seq2Seq(
    encoder,
    decoder,
    output_dim,
    horizon,
    optimizer=optim.RMSprop,
    optimizer_args={'lr': 0.01},
    scheduler=optim.lr_scheduler.StepLR,
    scheduler_args={'step_size': 5, 'gamma': 0.8},
)

/usr/local/Caskroom/miniconda/base/envs/capstone/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [12]:
count_parameters(model)

399617

In [13]:
output = model(input_tensor)

In [14]:
output.shape

torch.Size([64, 36, 1])

In [15]:
model.criterion(output, target_tensor)

tensor(1.1955, grad_fn=<MseLossBackward>)

In [16]:
trainer = Trainer(max_epochs=5)
trainer.fit(model, train_generator)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/usr/local/Caskroom/miniconda/base/envs/capstone/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 134 K 
1 | decoder | Decoder | 265 K 
------------------------------------
399 K     Trainable params
0         Non-trainable params
399 K     Total params
1.598     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/Caskroom/miniconda/base/envs/capstone/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/usr/local/Caskroom/miniconda/base/envs/capstone/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1